In [ ]:
from typing_extensions import Literal
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import convert_to_messages
from langchain_community.tools import DuckDuckGoSearchRun


In [ ]:
load_dotenv()

In [ ]:
openai_model = ChatOpenAI(model="gpt-4o")

In [ ]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
openai_model.invoke("Hi")

In [ ]:
web_search = DuckDuckGoSearchRun()

In [ ]:
web_search.invoke("What is the GDP of USA?")

In [ ]:
repl = PythonREPL()

In [ ]:
code = """
x = 5
y = x * 2
print(y)
"""

In [ ]:
repl.run(code)

In [ ]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """ Use this to execute python code. If you want to see the output of a value,
    you should print it out with 'print(...)'. This is visible to the User. """

    try:
        result = repl.run(code)

    except BaseException as e:
        return f"Failed to execute. Error: {repl(e)}"
    
    result_str = f"Successfully executed: \n'\'\'python\n{code}\n'\'\'\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [ ]:
def make_system_prompt(instruction: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{instruction}"
        )

In [ ]:
def get_next_node(last_message: BaseMessage, goto:str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto

In [ ]:
make_system_prompt(
    "You can only do research. You are working with a chart generator colleague."
)

In [ ]:
research_agent = create_react_agent(
    openai_model,
    tools=[web_search],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    )
)

In [ ]:
chart_agent = create_react_agent(
    openai_model,
    tools=[python_repl_tool],
    prompt=make_system_prompt(
        "You can only generate charts. You are working with a researcher colleague."
    )
)

In [ ]:
def research_node(state:MessagesState) -> Command[Literal["chart_generator", END]]:
    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "chart_generator")
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )
    return Command(
        update={
            "messages": result["messages"]
        },
        goto=goto
    )

In [ ]:
def chart_node(state: MessagesState) -> Command[Literal["researcher", END]]:
    result = chart_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "researcher")
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    return Command(
        update={
            # Share internal message history of chart agant with other agents
            "messages": result["messages"]
        },
        goto=goto
    )

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

workflow.add_edge(START, "researcher")
app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
input_1 = {"messages", [("user", "get the USA's GDP over the past 3 years. then make a line chart of it. Once you make the chart, Finish.")]}

In [ ]:
app.invoke(input_1)